In [1]:
from bs4 import BeautifulSoup as bs
import datetime, time
from datetime import date

import pandas as pd
import numpy as np

In [2]:
# 삼성전자 005930, 현대차 005830, 네이버 035420, KT&G 033780, 
code = {'sec': '005930', 'hyunmotor': '005830', 'naver': '035420', 'ktng': '033780'}

In [3]:
# column_eng = ['date', 'retail', 'foreigner', 'institutional', 
#                'security', 'investment_trust', 'bank', 'investment_bank', 'insurance', 'pension', 'others']
# column_kor = ['날짜', '개인', '외국인계', '기관계', '증권', '투신', '은행', '종금', '보험', '기금', '기타']
column_name = ['date', 'pp', 'fp', 'institutional', 
               'security', 'itp', 'bank', 'investment_bank', 'insurance', 'penp', 'others']

In [4]:
for key, val in code.items():
    filename = 'data/{}_{}.csv'.format('data', key)
    df_company = pd.read_csv(filename)
    df_company['date'] = df_company['date'].apply(lambda x : datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_company.set_index('date', inplace=True)
    url = '{}{}'.format('https://open.shinhaninvest.com/goodicyber/mk/1206.jsp?code=',val)
    df = pd.read_html(url, attrs={"class": "content1"}, flavor=["lxml", "bs4"])[0]
    df.columns = df.iloc[1] # column 명 변경
    df = df.drop([0,1]) # 필요없는 row 제거
    df.columns = column_name # replace with english column name
    reindex = np.arange(1,len(df)+1) # index를 일련번호로 대체하기 위하여 index 생성
    df.index = reindex # 새로운 인덱스로 대체
    df['date'] = df['date'].apply(lambda x : datetime.datetime.strptime(x, "%Y/%m/%d"))
    df_select = df[['date', 'pp', 'fp', 'itp', 'penp']]
    df_select.set_index('date', inplace=True)
    df_company.update(df_select, overwrite=False)
    df_company.reset_index(inplace=True)
    df_company.to_csv(filename, index=False)